In [ ]:
!pip install transformers
!pip install simpletransformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


English Data

### Mixing english and tamil translated to english data

In [3]:
import pandas as pd
concatenated=pd.read_csv('/content/drive/MyDrive/NLP-Project/BalencedLGBT_dataset.csv')
concatenated['category'] = concatenated['category'].replace('Non-anti-LGBT+ content', 'ally')
concatenated['category'] = concatenated['category'].replace('ally', 0)
concatenated['category'] = concatenated['category'].replace('Homophobic', 1)
concatenated['category'] = concatenated['category'].replace('Transphobic', 2)

In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (5331, 2)
Validation set shape: (1777, 2)
Test set shape: (1778, 2)


In [5]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

1    1795
0    1794
2    1742
Name: category, dtype: int64
0    598
1    598
2    581
Name: category, dtype: int64
0    599
1    598
2    581
Name: category, dtype: int64


In [6]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [7]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:

import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [9]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 32,
             "eval_batch_size": 32,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 2e-5,
             "manual_seed": 4}

# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [10]:
# Train the model
model.train_model(train_df)

  0%|          | 0/5331 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/167 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/167 [00:00<?, ?it/s]

(501, 0.3701009462158123)

Validation Set


In [11]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1777 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

{'mcc': 0.8802515294713106,
 'f1': 0.9203539865207832,
 'acc': 0.9200900393922341,
 'eval_loss': 0.2387403531133064}

Testing With English Data

In [12]:
import pandas as pd
e_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
e_test = e_test.rename(columns={'text                        ': 'text'})
#e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()
test_df = e_test
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [13]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who speak about culture., pre marita...",0,1
1,it's purely biological !,0,1
2,Everyone have there feeling please respect. We...,0,1
3,Just because it's made legal doesn't mean that...,0,1
4,audio problem waste video,0,1
...,...,...,...
65,Friendship s friendship.dont make it as sexual...,1,0
66,But according to hindu scriptures it is not. ...,0,1
67,Solute for everybady.....,0,1
68,aneesh cool a law that is created just in sp...,0,1


In [14]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.94      0.96       924
  homophobic       0.47      0.80      0.59        61
 transphobic       0.62      1.00      0.77         5

    accuracy                           0.93       990
   macro avg       0.69      0.91      0.77       990
weighted avg       0.95      0.93      0.94       990



### Checking Tamil data performance

In [15]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'tamil'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['tamil'] = combined_data['tamil'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,tamil,ln
0,0,Spr....2016 poitan feel happy with my partner ...,வசந்தம்....2016 போனேன் என் பார்ட்னருடன் மகிழ்ச...,eng_tam
1,0,R u still with ur partner,நீங்கள் இன்னும் உங்கள் துணையுடன் இருக்கிறீர்கள்,eng_tam
2,0,excellent movie..no unnecessary drama or scene...,அருமையான திரைப்படம்..தேவையற்ற நாடகம் அல்லது கா...,eng_tam
3,0,"For those who speak about culture., pre marita...","கலாச்சாரத்தைப் பற்றி பேசுபவர்களுக்கு, திருமணத்...",eng_tam
4,0,Best movie and people not understand relations...,சிறந்த திரைப்படம் மற்றும் மக்கள் உறவைப் புரிந்...,eng_tam
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,வருண் தனது தாயின் முக்கிய அன்பையும் நிழலையும் ...,eng_tam
986,0,i am really crying pro crying_face crying_fac...,நான் உண்மையிலேயே அழுகிறேன் ப்ரோ அழுகை_முகம் அழ...,eng_tam
987,0,They may be transgender but don't ever forgot ...,அவர்கள் திருநங்கைகளாக இருக்கலாம் ஆனால் அவர்களு...,eng_tam
988,0,It is their own choice. I support them No doub...,அது அவர்களின் சொந்த விருப்பம். நான் அவர்களை ஆத...,eng_tam


In [16]:

test = combined_data['eng'].tolist()
org = combined_data['tamil'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['tamil_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['tamil_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['tamil_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.8101010101010101


,category,eng,tamil,ln,eng_pred,tamil_pred
5,0,Awesome.... smiling_face_with_sunglasses ever...,அருமை.... கருப்புக் கண்ணாடியுடன்_சிரிக்கும்_மு...,eng_tam,0,1
6,0,99% tamilans cant understand r accept it becau...,99% இத்தாலியர்களால் அதைப் புரிந்து கொள்ளவோ ​​அ...,eng_tam,0,1
12,0,Bold speech. I support you sister face_savori...,துணிச்சலான பேச்சு. நான் உங்களை ஆதரிக்கிறேன் சக...,eng_tam,0,1
21,0,My point of you it's correct because there all...,"உங்கள் எண்ணம் சரியானது, ஏனென்றால் எல்லா உணர்வு...",eng_tam,0,1
28,0,"Madan sir, pls make a video about the origins ...","மதன் சார், விசித்திரக் கதைகள் அல்லது கிரிம்ஸின...",eng_tam,0,1
...,...,...,...,...,...,...
971,0,Did u know that women experience more pleasure...,பெண்கள் மற்ற பெண்களுடன் உடலுறவு கொள்ளும்போது அ...,eng_tam,0,1
978,0,She is innocent nd cute,அவள் அப்பாவி மற்றும் அழகானவள்,eng_tam,0,2
979,0,She is beautiful,அவள் அழகாக இருக்கிறாள்,eng_tam,0,2
983,0,Hats off handshake .... Semma semma finally t...,ஹேட்ஸ் ஆஃப் ஹேண்ட்ஷேக்.... செம்ம செம்ம கடைசியி...,eng_tam,0,1


In [17]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['tamil_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.79      0.87       924
  homophobic       0.19      0.64      0.30        61
 transphobic       0.06      0.40      0.11         5

    accuracy                           0.78       990
   macro avg       0.41      0.61      0.43       990
weighted avg       0.92      0.78      0.83       990



In [18]:
test = combined_data['tamil'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"கலாச்சாரத்தைப் பற்றி பேசுபவர்களுக்கு, திருமணத்...",0,1
1,அருமை.... கருப்புக் கண்ணாடியுடன்_சிரிக்கும்_மு...,0,1
2,99% இத்தாலியர்களால் அதைப் புரிந்து கொள்ளவோ ​​அ...,0,1
3,துணிச்சலான பேச்சு. நான் உங்களை ஆதரிக்கிறேன் சக...,0,1
4,"உங்கள் எண்ணம் சரியானது, ஏனென்றால் எல்லா உணர்வு...",0,1
...,...,...,...
211,பெண்கள் மற்ற பெண்களுடன் உடலுறவு கொள்ளும்போது அ...,0,1
212,அவள் அப்பாவி மற்றும் அழகானவள்,0,2
213,அவள் அழகாக இருக்கிறாள்,0,2
214,ஹேட்ஸ் ஆஃப் ஹேண்ட்ஷேக்.... செம்ம செம்ம கடைசியி...,0,1


# Training with english and testing with Hindi

In [19]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'hindi'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['hindi'] = combined_data['hindi'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,hindi,ln
0,0,Spr....2016 poitan feel happy with my partner ...,Spr....2016 कवि अपने पार्टनर के साथ खुश हैं,eng_hin
1,0,R u still with ur partner,क्या आप अभी भी अपने साथी के साथ हैं,eng_hin
2,0,excellent movie..no unnecessary drama or scene...,उत्कृष्ट फिल्म..कोई अनावश्यक नाटक या दृश्य नही...,eng_hin
3,0,"For those who speak about culture., pre marita...","संस्कृति की बात करने वालों के लिए, शादी से पहल...",eng_hin
4,0,Best movie and people not understand relations...,बेहतरीन फिल्म और लोग रिश्ते को नहीं समझते हैं ...,eng_hin
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,ऐसा लगता है कि कार्तिक ने वरुण पर फायदा उठाया ...,eng_hin
986,0,i am really crying pro crying_face crying_fac...,मैं सच में रो रहा हूँ चेहरा रोना_चेहरा रोना_चे...,eng_hin
987,0,They may be transgender but don't ever forgot ...,वो ट्रांसजेंडर हो सकते हैं लेकिन ये कभी नहीं भ...,eng_hin
988,0,It is their own choice. I support them No doub...,यह उनकी अपनी पसंद है। मैं उनका समर्थन करता हूं...,eng_hin


In [20]:

test = combined_data['eng'].tolist()
org = combined_data['hindi'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['hindi_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['hindi_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['hindi_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.794949494949495


,category,eng,hindi,ln,eng_pred,hindi_pred
5,0,Awesome.... smiling_face_with_sunglasses ever...,बहुत बढ़िया.... धूप के चश्मे के साथ_मुस्कान_मु...,eng_hin,0,1
6,0,99% tamilans cant understand r accept it becau...,99% इटालियंस इसे समझ या स्वीकार नहीं कर सकते क...,eng_hin,0,1
13,0,Really this prank will be more powerful Feedba...,वास्तव में यह शरारत समाज से अधिक शक्तिशाली प्र...,eng_hin,0,1
26,0,She is beautiful smiling_face_with_heart-eyes...,वह आशिक के साथ मुस्कुराती हुई खूबसूरत है,eng_hin,0,1
32,0,Woman is a beautiful creation on earth...which...,औरत धरती पर एक खूबसूरत रचना है...जो किसी को भी...,eng_hin,0,1
...,...,...,...,...,...,...
970,0,It is right 100. It is their life and their ch...,यह सही 100 है। यह उनका जीवन और उनकी पसंद है। म...,eng_hin,0,1
975,0,Really great akka,वाकई बढ़िया अक्का,eng_hin,0,2
980,0,4 arivu athiham bro... Arivey illathavan kudut...,4 अरिवु अथिहम ब्रो... अरिवे इल्लतावन कुदुथा थी...,eng_hin,0,2
985,0,Looks like Karthik took advantage on Varun whe...,ऐसा लगता है कि कार्तिक ने वरुण पर फायदा उठाया ...,eng_hin,0,1


In [21]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['hindi_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.77      0.86       924
  homophobic       0.20      0.72      0.31        61
 transphobic       0.00      0.00      0.00         5

    accuracy                           0.76       990
   macro avg       0.39      0.50      0.39       990
weighted avg       0.92      0.76      0.82       990



In [22]:
test = combined_data['hindi'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"संस्कृति की बात करने वालों के लिए, शादी से पहल...",0,1
1,बहुत बढ़िया.... धूप के चश्मे के साथ_मुस्कान_मु...,0,1
2,99% इटालियंस इसे समझ या स्वीकार नहीं कर सकते क...,0,1
3,वास्तव में यह शरारत समाज से अधिक शक्तिशाली प्र...,0,1
4,वह आशिक के साथ मुस्कुराती हुई खूबसूरत है,0,1
...,...,...,...
228,यह सही 100 है। यह उनका जीवन और उनकी पसंद है। म...,0,1
229,वाकई बढ़िया अक्का,0,2
230,4 अरिवु अथिहम ब्रो... अरिवे इल्लतावन कुदुथा थी...,0,2
231,ऐसा लगता है कि कार्तिक ने वरुण पर फायदा उठाया ...,0,1


# Training with english and testing with Telugu

In [23]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'tel'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['tel'] = combined_data['tel'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,tel,ln
0,0,Spr....2016 poitan feel happy with my partner ...,వసంతం....2016 పోయిటన్ నా భాగస్వామితో సంతోషంగా ...,eng_tel
1,0,R u still with ur partner,మీరు ఇప్పటికీ మీ భాగస్వామితో ఉన్నారు,eng_tel
2,0,excellent movie..no unnecessary drama or scene...,అద్భుతమైన సినిమా..అనవసరమైన డ్రామా లేదా సన్నివే...,eng_tel
3,0,"For those who speak about culture., pre marita...","సంస్కృతి గురించి మాట్లాడే వారికి, వివాహానికి మ...",eng_tel
4,0,Best movie and people not understand relations...,ఉత్తమ చిత్రం మరియు వ్యక్తులు సంబంధాన్ని అర్థం ...,eng_tel
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,కార్తీక్ వరుణ్ తన తల్లి నుండి తన ప్రధాన ప్రేమన...,eng_tel
986,0,i am really crying pro crying_face crying_fac...,నేను నిజంగా ఏడుస్తున్నాను ప్రో క్రయింగ్_ఫేస్ క...,eng_tel
987,0,They may be transgender but don't ever forgot ...,వారు లింగమార్పిడి కావచ్చు కానీ వారు కూడా మనుషు...,eng_tel
988,0,It is their own choice. I support them No doub...,అది వారి స్వంత ఎంపిక. నేను వారికి మద్దతు ఇస్తు...,eng_tel


In [24]:

test = combined_data['eng'].tolist()
org = combined_data['tel'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['tel_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['tel_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['tel_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.8404040404040404


,category,eng,tel,ln,eng_pred,tel_pred
6,0,99% tamilans cant understand r accept it becau...,99% ఇటాలియన్లు దీనిని అర్థం చేసుకోలేరు లేదా అం...,eng_tel,0,1
24,0,It is good that everyone understands the feeli...,మనం కూడా ఈ సమాజంలో ప్రశాంతంగా జీవించాలంటే మనలా...,eng_tel,0,1
31,0,I mean...there was an attempt?? Meanwhile I'm ...,అంటే... ప్రయత్నం జరిగిందా?? ఇంతలో నేను అందమైన ...,eng_tel,0,1
32,0,Woman is a beautiful creation on earth...which...,స్త్రీ భూమిపై అందమైన సృష్టి...ఎవరికైనా అందంగా ...,eng_tel,0,1
36,0,Can you share your thoughts on cannabis and th...,మీరు గంజాయి మరియు THC 'tetrahydrocannabinol సమ...,eng_tel,0,1
...,...,...,...,...,...,...
967,0,Jugal Romil of course me too ...,జుగల్ రోమిల్ నేను కూడా,eng_tel,1,0
978,0,She is innocent nd cute,ఆమె అమాయకంగా మరియు అందమైనది,eng_tel,0,1
980,0,4 arivu athiham bro... Arivey illathavan kudut...,4 అరివు అతిహమ్ బ్రో... అరివే ఇల్లతవన్ కుడుత తీ...,eng_tel,0,1
987,0,They may be transgender but don't ever forgot ...,వారు లింగమార్పిడి కావచ్చు కానీ వారు కూడా మనుషు...,eng_tel,0,1


In [25]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['tel_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.84      0.90       924
  homophobic       0.22      0.64      0.33        61
 transphobic       0.07      0.20      0.11         5

    accuracy                           0.82       990
   macro avg       0.42      0.56      0.44       990
weighted avg       0.92      0.82      0.86       990



In [26]:
test = combined_data['tel'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"సంస్కృతి గురించి మాట్లాడే వారికి, వివాహానికి మ...",0,1
1,99% ఇటాలియన్లు దీనిని అర్థం చేసుకోలేరు లేదా అం...,0,1
2,మనం కూడా ఈ సమాజంలో ప్రశాంతంగా జీవించాలంటే మనలా...,0,1
3,అంటే... ప్రయత్నం జరిగిందా?? ఇంతలో నేను అందమైన ...,0,1
4,స్త్రీ భూమిపై అందమైన సృష్టి...ఎవరికైనా అందంగా ...,0,1
...,...,...,...
169,అనీష్ కూల్ కేవలం 70 ఏళ్లలో రూపొందించిన చట్టం క...,0,1
170,ఆమె అమాయకంగా మరియు అందమైనది,0,1
171,4 అరివు అతిహమ్ బ్రో... అరివే ఇల్లతవన్ కుడుత తీ...,0,1
172,వారు లింగమార్పిడి కావచ్చు కానీ వారు కూడా మనుషు...,0,1


# Training with english and testing with Malyalam

In [27]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'mal'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['mal'] = combined_data['mal'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,mal,ln
0,0,Spr....2016 poitan feel happy with my partner ...,Spr....2016 പോയതാൻ എന്റെ പങ്കാളിയുമായി സന്തോഷമ...,eng_ml
1,0,R u still with ur partner,നിങ്ങൾ ഇപ്പോഴും നിങ്ങളുടെ പങ്കാളിക്കൊപ്പമാണ്,eng_ml
2,0,excellent movie..no unnecessary drama or scene...,മികച്ച സിനിമ.. അനാവശ്യ നാടകങ്ങളോ രംഗങ്ങളോ ഇല്ല...,eng_ml
3,0,"For those who speak about culture., pre marita...","സംസ്കാരത്തെക്കുറിച്ച് സംസാരിക്കുന്നവർക്ക്, വിവ...",eng_ml
4,0,Best movie and people not understand relations...,മികച്ച സിനിമയും ആളുകൾക്ക് ബന്ധം മനസ്സിലാകാത്തത...,eng_ml
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,അമ്മയിൽ നിന്നുള്ള പ്രധാന സ്നേഹവും നിഴലുകളും നഷ...,eng_ml
986,0,i am really crying pro crying_face crying_fac...,ഞാൻ ശരിക്കും കരയുന്നു പ്രോ കരയുന്നു_മുഖം കരയുന...,eng_ml
987,0,They may be transgender but don't ever forgot ...,അവർ ട്രാൻസ്‌ജെൻഡർ ആയിരിക്കാം പക്ഷെ അവരും മനുഷ്...,eng_ml
988,0,It is their own choice. I support them No doub...,അത് അവരുടെ സ്വന്തം തിരഞ്ഞെടുപ്പാണ്. ഞാൻ അവരെ പ...,eng_ml


In [28]:

test = combined_data['eng'].tolist()
org = combined_data['mal'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['mal_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['mal_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['mal_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.7222222222222222


,category,eng,mal,ln,eng_pred,mal_pred
6,0,99% tamilans cant understand r accept it becau...,99% ഇറ്റലിക്കാർക്കും ഇത് മനസ്സിലാക്കാനോ അംഗീകര...,eng_ml,0,1
9,0,Ak Be The Change I don't think it's good ..it...,Ak Be The Change ഇത് നല്ലതാണെന്ന് എനിക്ക് തോന...,eng_ml,0,1
19,0,oh jealous???? then u also call ur self as que...,അയ്യോ അസൂയ???? അപ്പോൾ നിങ്ങൾ സ്വയം രാജ്ഞി എന്ന...,eng_ml,0,1
21,0,My point of you it's correct because there all...,നിങ്ങളെക്കുറിച്ചുള്ള എന്റെ അഭിപ്രായം അത് ശരിയാ...,eng_ml,0,1
24,0,It is good that everyone understands the feeli...,നമുക്കും ഈ സമൂഹത്തിൽ സമാധാനമായി ജീവിക്കാൻ ഞങ്ങ...,eng_ml,0,1
...,...,...,...,...,...,...
962,0,Solute for everybady.....,എല്ലാവർക്കും പരിഹാരം.....,eng_ml,1,0
967,0,Jugal Romil of course me too ...,ജുഗൽ റോമിൽ തീർച്ചയായും ഞാനും,eng_ml,1,0
972,0,How u r earning subscribers without video but ...,വീഡിയോ ഇല്ലാതെ നിങ്ങൾ എങ്ങനെ സബ്‌സ്‌ക്രൈബർമാരെ...,eng_ml,0,1
985,0,Looks like Karthik took advantage on Varun whe...,അമ്മയിൽ നിന്നുള്ള പ്രധാന സ്നേഹവും നിഴലുകളും നഷ...,eng_ml,0,1


In [29]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['mal_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.98      0.69      0.81       924
  homophobic       0.16      0.77      0.26        61
 transphobic       0.05      0.40      0.10         5

    accuracy                           0.70       990
   macro avg       0.40      0.62      0.39       990
weighted avg       0.92      0.70      0.78       990



In [30]:
test = combined_data['mal'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"സംസ്കാരത്തെക്കുറിച്ച് സംസാരിക്കുന്നവർക്ക്, വിവ...",0,1
1,99% ഇറ്റലിക്കാർക്കും ഇത് മനസ്സിലാക്കാനോ അംഗീകര...,0,1
2,Ak Be The Change ഇത് നല്ലതാണെന്ന് എനിക്ക് തോന...,0,1
3,അയ്യോ അസൂയ???? അപ്പോൾ നിങ്ങൾ സ്വയം രാജ്ഞി എന്ന...,0,1
4,നിങ്ങളെക്കുറിച്ചുള്ള എന്റെ അഭിപ്രായം അത് ശരിയാ...,0,1
...,...,...,...
294,എന്നാൽ ഹിന്ദു ഗ്രന്ഥങ്ങൾ പ്രകാരം അങ്ങനെയല്ല.,0,1
295,"അനീഷ് കൂൾ, 70 വർഷത്തിനുള്ളിൽ സൃഷ്ടിക്കപ്പെട്ട ...",0,1
296,വീഡിയോ ഇല്ലാതെ നിങ്ങൾ എങ്ങനെ സബ്‌സ്‌ക്രൈബർമാരെ...,0,1
297,അമ്മയിൽ നിന്നുള്ള പ്രധാന സ്നേഹവും നിഴലുകളും നഷ...,0,1


Here we rephrased few sentences and tested on our model to analysie its performance

Robustness test

## Robustness Analysis

In [31]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Synonymn aug

In [32]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import random

data = test_df.copy()
syn_aug = naw.SynonymAug(aug_src='wordnet')
lis = []
for index, row in data.iterrows():
    text = syn_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . 2016 poitan tactile property happy w...
1,R u still with ur partner,0,Roentgen u still with ur better half
2,excellent movie..no unnecessary drama or scene...,0,excellent movie. . no unnecessary dramatic pla...
3,"For those who speak about culture., pre marita...",0,"For those who speak about culture. , pre marit..."
4,Best movie and people not understand relations...,0,Best movie and citizenry non understand relati...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...
986,i am really crying pro crying_face crying_fac...,0,i am really crying pro crying_face crying_face...
987,They may be transgender but don't ever forgot ...,0,They whitethorn comprise transgender merely do...
988,It is their own choice. I support them No doub...,0,It is their ain selection. I hold them No doub...


In [33]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.793939393939394


,text,labels,Changed_text,Changed_text_labels,text_labels
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . 2016 poitan tactile property happy w...,1,0
6,99% tamilans cant understand r accept it becau...,0,99% tamilans cant understand r accept it becau...,1,0
7,U r very clean talker... Body for mud... Only ...,0,U r real clean utterer. .. Eubstance for clay....,1,0
9,Ak Be The Change I don't think it's good ..it...,0,Ak Glucinium The Change Atomic number 53 get i...,1,0
12,Bold speech. I support you sister face_savori...,0,Bold speech. Atomic number 53 patronise you si...,1,0
...,...,...,...,...,...
968,I am crying ...,0,1 equal crying. ..,1,0
970,It is right 100. It is their life and their ch...,0,Information technology is veracious 100. Infor...,1,0
974,how about me? do i look transgenders? ...,0,how about maine? act i look transgenders?,2,0
978,She is innocent nd cute,0,She be barren nd cute,1,0


In [34]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Spr. .. . 2016 poitan tactile property happy w...,0,1
1,"For those who speak about culture. , pre marit...",0,1
2,99% tamilans cant understand r accept it becau...,0,1
3,U r real clean utterer. .. Eubstance for clay....,0,1
4,Ak Glucinium The Change Atomic number 53 get i...,0,1
...,...,...,...
245,1 equal crying. ..,0,1
246,Information technology is veracious 100. Infor...,0,1
247,how about maine? act i look transgenders?,0,2
248,She be barren nd cute,0,1


In [35]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.74      0.85       924
  homophobic       0.21      0.90      0.34        61
 transphobic       0.11      1.00      0.20         5

    accuracy                           0.75       990
   macro avg       0.44      0.88      0.46       990
weighted avg       0.94      0.75      0.81       990



## RandomCharAug

In [36]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import random

data = test_df.copy()
char_aug = nac.RandomCharAug(action="insert")
lis = []
for index, row in data.iterrows():
    text = char_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . 2016 zpoi4tan lfeeel happy &wijth my...
1,R u still with ur partner,0,R u 6sKtill whitph ur partner
2,excellent movie..no unnecessary drama or scene...,0,excellent movie. . no unnecessary d$ram+a or s...
3,"For those who speak about culture., pre marita...",0,"For those who spQea%k uaboyut cufl0tur%e. , pr..."
4,Best movie and people not understand relations...,0,Best movYiLe and (peocple not understand SrQel...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik tTohok advoantJagHe on Varu...
986,i am really crying pro crying_face crying_fac...,0,i am reaHlley cdrvying pro crying_face cr2y9il...
987,They may be transgender but don't ever forgot ...,0,nTheQy may be tQra9nsgenVdIer but don ' t ever...
988,It is their own choice. I support them No doub...,0,It is @theixr own cShoiMce. I supBpoGr)t tvhme...


In [37]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.8818181818181818


,text,labels,Changed_text,Changed_text_labels,text_labels
7,U r very clean talker... Body for mud... Only ...,0,U r )verwy LcleaIn zt+alker. .. BoIdKy for mud...,1,0
36,Can you share your thoughts on cannabis and th...,0,Can you lshar%e ybouGr Rt!hougbhts on cannabis...,1,0
54,Royal Queen You must know that laws are also ...,0,RFoyajl Queen You mousgt know otrhat laws are ...,1,0
60,Both of u who replied to my comment thanks for...,1,Both of u who %rdepClied to my comment thanks ...,0,1
62,The person who asking completely doesn't know...,0,The person who zaskinRg completely doesn ' t k...,1,0
...,...,...,...,...,...
954,Give equilism to all human being. slightly_sm...,0,Give 7equjiblism to all hZumaan being. sligNht...,1,0
958,But according to hindu scriptures it is not. ...,0,But aDccBoJrding to h$i2ndu sAcrisp%tures it i...,0,1
967,Jugal Romil of course me too ...,0,Jug7aYl Romil of scVourse me too,0,1
976,After living in western countries Iam very sur...,0,After clriving in western countBrxieTs Iam 1ve...,1,0


In [38]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who spQea%k uaboyut cufl0tur%e. , pr...",0,1
1,U r )verwy LcleaIn zt+alker. .. BoIdKy for mud...,0,1
2,Can you lshar%e ybouGr Rt!hougbhts on cannabis...,0,1
3,it ' s zpur5ely bYiologixctal!,0,1
4,RFoyajl Queen You mousgt know otrhat laws are ...,0,1
...,...,...,...
110,Give 7equjiblism to all hZumaan being. sligNht...,0,1
111,Soluht0e for 9eTverybad1y. .. ..,0,1
112,aneesh lcoOol a law WtDhat is RcreaktVed just ...,0,1
113,After clriving in western countBrxieTs Iam 1ve...,0,1


In [39]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.91      0.94       924
  homophobic       0.29      0.54      0.38        61
 transphobic       0.17      0.20      0.18         5

    accuracy                           0.88       990
   macro avg       0.47      0.55      0.50       990
weighted avg       0.92      0.88      0.90       990



## Sentence suffiling

In [40]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
#import nlpaug.augmenter.sentence as nas
import random
import nltk


# Create an instance of the augmenter class
data = test_df.copy()
tokenizer = nltk.tokenize.sent_tokenize
sen_aug = naw.RandomWordAug(action="swap", tokenizer=tokenizer)
lis = []
for index, row in data.iterrows():
    text = sen_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,Spr....2016 poitan feel happy with my partner
1,R u still with ur partner,0,R u still with ur partner
2,excellent movie..no unnecessary drama or scene...,0,Looking forward fr more movies frm U. excellen...
3,"For those who speak about culture., pre marita...",0,"For those who speak about culture., pre marita..."
4,Best movie and people not understand relations...,0,Best movie and people not understand relations...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...
986,i am really crying pro crying_face crying_fac...,0,i am really crying pro crying_face crying_fac...
987,They may be transgender but don't ever forgot ...,0,They may be transgender but don't ever forgot ...
988,It is their own choice. I support them No doub...,0,It is their own choice. They are also humans a...


In [42]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.9848484848484849


,text,labels,Changed_text,Changed_text_labels,text_labels
36,Can you share your thoughts on cannabis and th...,0,And the benefit from it widely healing many ma...,1,0
78,Thank you for your positive response. The film...,0,Thank you for your positive response. So we ca...,1,0
327,next 10 years'la total world people Hiv patien...,1,(no children). next 10 years'la total world pe...,0,1
370,Intha ulagathula normal ah irukuravanga kitta ...,0,This is the major difference in this society.....,1,0
397,swetha k seri tell me what physical changes th...,0,Are they any genetic changes seen? swetha k se...,1,0
452,"First of all, the interviewer is not asking th...",0,They have just declared mutual gay sex is not ...,1,0
478,Nakeeran please dont promote these kind of int...,1,Nakeeran please dont promote these kind of int...,0,1
544,"Though I don't support same sex marriages , I ...",1,There's a famous quote by a female non English...,0,1
548,The one who r saying it's wrong it's shit it's...,0,Hey we r not forcing u guys to be homosexual n...,1,0
678,Govt pls provide some job for those ppl.. so t...,0,Politicians: instead of blaming among urself b...,0,1


In [43]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who speak about culture., pre marita...",0,1
1,And the benefit from it widely healing many ma...,0,1
2,it's purely biological !,0,1
3,Come out of the box and respect everyone. ever...,0,1
4,Just because it's made legal doesn't mean that...,0,1
...,...,...,...
72,Slightly_smiling_face give equilism to all hum...,0,1
73,But according to hindu scriptures it is not.,0,1
74,Solute for everybady.....,0,1
75,aneesh cool a law that is created just in sp...,0,1


In [44]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.93      0.96       924
  homophobic       0.44      0.79      0.56        61
 transphobic       0.62      1.00      0.77         5

    accuracy                           0.92       990
   macro avg       0.68      0.91      0.76       990
weighted avg       0.95      0.92      0.93       990



naw.RandomWordAug(action='delete')

In [45]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.sentence as nas
import random

data = test_df.copy()
w_aug = naw.RandomWordAug(action='delete')
lis = []
for index, row in data.iterrows():
    text = w_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,.. .. 2016 feel happy my
1,R u still with ur partner,0,R still with partner
2,excellent movie..no unnecessary drama or scene...,0,movie. . unnecessary drama or. . was natural. ...
3,"For those who speak about culture., pre marita...",0,"For those who speak about culture. , pre marit..."
4,Best movie and people not understand relations...,0,Best movie and understand feeling I my life
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Like Karthik took advantage on Varun loses his...
986,i am really crying pro crying_face crying_fac...,0,i am really crying_face crying_face loudly_cry...
987,They may be transgender but don't ever forgot ...,0,May be transgender don ' t that also humans! !...
988,It is their own choice. I support them No doub...,0,It is. I them about. They also humans and righ...


In [46]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

similarity in predection score: 0.8929292929292929


,text,labels,Changed_text,Changed_text_labels,text_labels
14,She has a point,0,She point,2,0
27,Super anna..Verra level..All are have red bloo...,0,Anna. . Verra. . blood. .. . the Human beings..,1,0
28,"Madan sir, pls make a video about the origins ...",0,"Madan, make a about the origins tales grimm ' ...",1,0
31,I mean...there was an attempt?? Meanwhile I'm ...,0,I. .. ?? Meanwhile I ' m marveling at skin.,1,0
36,Can you share your thoughts on cannabis and th...,0,You share your thoughts on and the ' thetrahyd...,1,0
...,...,...,...,...,...
948,Kayathri Devi what do you mean by that? or you...,0,Devi what you mean that? or you still can ' ac...,1,0
962,Solute for everybady.....,0,For. .. ..,0,1
967,Jugal Romil of course me too ...,0,Jugal Romil of me,0,1
978,She is innocent nd cute,0,She innocent nd,1,0


In [47]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who speak about culture. , pre marit...",0,1
1,She point,0,2
2,Anna. . Verra. . blood. .. . the Human beings..,0,1
3,"Madan, make a about the origins tales grimm ' ...",0,1
4,I. .. ?? Meanwhile I ' m marveling at skin.,0,1
...,...,...,...
130,Devi what you mean that? or you still can ' ac...,0,1
131,But according scriptures it is.,0,1
132,aneesh cool a law created in span of 70 years ...,0,1
133,She innocent nd,0,1


In [48]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.98      0.87      0.92       924
  homophobic       0.29      0.72      0.41        61
 transphobic       0.21      0.60      0.32         5

    accuracy                           0.86       990
   macro avg       0.49      0.73      0.55       990
weighted avg       0.93      0.86      0.89       990



##Analysing sentence wise

In [49]:

sentences = [
"homophobic are individuals who support and advocate for marginalized communities, even if they do not belong to those communities themselves.",
"Being an ally requires more than just a willingness to help - it requires active listening, learning, and action.",
"Allies can play a critical role in advancing social justice and creating more equitable societies.",
"Allies should always prioritize the needs and voices of marginalized communities over their own.",
"Allies should use their privilege to uplift and amplify the voices of marginalized individuals.",
"Allies should be willing to engage in uncomfortable conversations and challenge their own biases and assumptions.",
"homophobic is the irrational fear or hatred of transgender people.",
"Discrimination against transgender individuals is a form of transphobia.",
"Using the wrong pronouns to refer to a transgender person is an act of transphobia.",
"Transphobia can have serious negative impacts on the mental health of transgender individuals.",
"Allies of the transgender community should work to combat transphobia wherever they encounter it.",
"Transphobia is a form of prejudice that has no place in a just and equitable society.",
"Transphobia often stems from a lack of understanding or empathy for the experiences of transgender individuals.",
"Transphobia can manifest in many different ways, from physical violence to verbal harassment to exclusion from certain spaces.",
"Transphobia is not just a problem for transgender individuals, but for society as a whole.",
"By standing up against transphobia, we can create a world where everyone is accepted for who they are.",
]
labels = [1,0,0,0,0,0,1,2,2,2,2,2,2,2,2,2]


test = sentences
labels = labels
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
mis

,text,true_label,predicted_label
0,Being an ally requires more than just a willin...,0,1
1,Allies can play a critical role in advancing s...,0,1
2,Allies should use their privilege to uplift an...,0,1
3,Allies should be willing to engage in uncomfor...,0,1
4,homophobic is the irrational fear or hatred of...,1,0
5,Discrimination against transgender individuals...,2,0
6,Using the wrong pronouns to refer to a transge...,2,0
7,Transphobia can have serious negative impacts ...,2,0
8,Allies of the transgender community should wor...,2,0
9,Transphobia is a form of prejudice that has no...,2,1


In [51]:

sentences = [
"homophobic are individuals who support and advocate for marginalized communities, even if they do not belong to those communities themselves.",
"Being an ally requires more than just a willingness to help - it requires active listening, learning, and action.",
"Allies can play a critical role in advancing social justice and creating more equitable societies.",
"Allies should always prioritize the needs and voices of marginalized communities over their own.",
"Allies should use their privilege to uplift and amplify the voices of marginalized individuals.",
"Allies should be willing to engage in uncomfortable conversations and challenge their own biases and assumptions.",
"homophobic is the irrational fear or hatred of transgender people.",
"Discrimination against transgender individuals is a form of transphobia.",
"Using the wrong pronouns to refer to a transgender person is an act of transphobia.",
"Transphobia can have serious negative impacts on the mental health of transgender individuals.",
"Allies of the transgender community should work to combat transphobia wherever they encounter it.",
"Transphobia is a form of prejudice that has no place in a just and equitable society.",
"Transphobia often stems from a lack of understanding or empathy for the experiences of transgender individuals.",
"Transphobia can manifest in many different ways, from physical violence to verbal harassment to exclusion from certain spaces.",
"Transphobia is not just a problem for transgender individuals, but for society as a whole.",
"By standing up against transphobia, we can create a world where everyone is accepted for who they are.",
]
labels = [1,0,0,0,0,0,1,2,2,2,2,2,2,2,2,2]


test = sentences
labels = labels
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Being an ally requires more than just a willin...,0,1
1,Allies can play a critical role in advancing s...,0,1
2,Allies should use their privilege to uplift an...,0,1
3,Allies should be willing to engage in uncomfor...,0,1
4,homophobic is the irrational fear or hatred of...,1,0
5,Discrimination against transgender individuals...,2,0
6,Using the wrong pronouns to refer to a transge...,2,0
7,Transphobia can have serious negative impacts ...,2,0
8,Allies of the transgender community should wor...,2,0
9,Transphobia is a form of prejudice that has no...,2,1


In [52]:



sentences = [
    
"Homophobia is the irrational fear or hatred of LGBTQ+ individuals, specifically those who are attracted to people of the same gender.",
"Homophobia can take many forms, including bullying, harassment, discrimination, and violence.",
"Homophobia can have serious negative impacts on the mental health and well-being of LGBTQ+ individuals.",
"Allies of the LGBTQ+ community should work to combat homophobia wherever they encounter it.",
"Homophobia is a form of prejudice that has no place in a just and equitable society.",
"Homophobia often stems from a lack of understanding or empathy for the experiences of LGBTQ+ individuals.",
"Homophobia can manifest in many different ways, from derogatory language to physical violence to exclusion from certain spaces.",
"Homophobia is not just a problem for LGBTQ+ individuals, but for society as a whole.",
"By standing up against homophobia, we can create a world where everyone is free to love who they choose.",
"Homophobia is not only harmful, it is also illogical and unjustified."
]
labels = [1,1,1,0,1,1,1,1,1,1]


test = sentences
labels = labels
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"By standing up against homophobia, we can crea...",1,0
